In [4]:
# Cell 1: Setup paths and import dependencies
import sys
import os
from collections import deque, defaultdict

# Add parent directory to path to find YOLOv7 modules
sys.path.append('..')
sys.path.append(os.path.abspath('..'))  # Absolute path to be extra safe

# Import dependencies
import numpy as np
import cv2
import torch
import time
import math
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

print("Setup complete. YOLOv7 modules imported successfully.")

Setup complete. YOLOv7 modules imported successfully.


In [5]:
# Cell 2: Set up the device and load model
# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define paths for YOLOv7-W6-Pose weights
# Try multiple potential locations
potential_paths = [
    '../yolov7-w6-pose.pt',  # Root directory
    'yolov7-w6-pose.pt',     # Current directory
    '../models/yolov7-w6-pose.pt' # Models directory
]

# Find the first path that exists
model_path = None
for path in potential_paths:
    if os.path.exists(path):
        model_path = path
        break

# If model not found, attempt to download
if model_path is None:
    download_path = '../yolov7-w6-pose.pt'  # Download to root directory
    print(f"YOLOv7-W6-Pose weights not found. Downloading to {download_path}...")
    
    # GitHub release URL for YOLOv7-W6-Pose weights
    url = 'https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt'
    
    try:
        import requests
        print(f"Downloading {url}...")
        response = requests.get(url)
        with open(download_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded to {download_path}")
        model_path = download_path
    except Exception as e:
        print(f"Download failed: {e}")
        print("Please download the weights manually from:")
        print("https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt")
        raise FileNotFoundError("Model weights not found and download failed")

print(f"Loading model from {model_path}")

# Load YOLOv7-pose model
weights = torch.load(model_path, map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)
    
print("Model loaded successfully")

Using device: cuda:0
Loading model from ../yolov7-w6-pose.pt
Model loaded successfully


In [6]:
# Cell 3: Define the FallDetector class
class FallDetector:
    def __init__(self):
        # Threshold parameters based on the paper
        self.LENGTH_FACTOR_ALPHA = 0.5  # ALPHA value from paper
        self.VELOCITY_THRESHOLD = 0.5   # SPEED_THRESHOLD from paper
        self.ANGLE_THRESHOLD = 45       # ANGLE_THRESHOLD from paper
        self.TORSO_ANGLE_THRESHOLD = 50  # For torso orientation
        self.CONFIDENCE_THRESHOLD = 0.5  # Confidence threshold for keypoints
        self.TARGET_FPS = 25            # Target FPS for processing
        
        # State tracking with buffers for smoothing (as in the shared code)
        self.prev_keypoints = None
        self.velocity_buffer = deque(maxlen=3)  # Buffer for velocity smoothing
        self.fall_buffer = deque(maxlen=2)      # Buffer for fall detection smoothing
        self.prev_frame_time = None
        self.fall_start_time = None
        self.prev_shoulder_y = None
        self.current_state = "normal"  # State tracking: "normal", "falling", "fallen"

    def calculate_angle(self, a, b, c):
        """Calculate angle between three points (used for joint angles)."""
        try:
            ba = np.array([a[0]-b[0], a[1]-b[1]])
            bc = np.array([c[0]-b[0], c[1]-b[1]])
            cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
            return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
        except:
            return 180

    def calculate_torso_angle(self, shoulders, hips):
        """Calculate angle of torso relative to vertical."""
        shoulder_center = np.mean(shoulders, axis=0)
        hip_center = np.mean(hips, axis=0)
        vertical_vector = np.array([0, 1])
        torso_vector = np.array([hip_center[0]-shoulder_center[0], hip_center[1]-shoulder_center[1]])
        
        if np.linalg.norm(torso_vector) < 1e-6:
            return 90
            
        cosine = np.dot(torso_vector, vertical_vector) / (np.linalg.norm(torso_vector) + 1e-6)
        return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

    def detect_fall(self, keypoints):
        """
        Detect falls using keypoints from YOLOv7-W6-Pose.
        Implements the approach described in the paper with smoothing buffers.
        
        Args:
            keypoints: Keypoints array from YOLOv7-W6-Pose model
            
        Returns:
            is_fall: Boolean indicating if a fall is detected
            current_state: Current state of the fall detection state machine
            conditions: List of conditions that contributed to the fall detection
        """
        # Keypoint indices based on YOLOv7-W6-Pose output
        NOSE = 0
        LEFT_SHOULDER = 5
        RIGHT_SHOULDER = 6
        LEFT_HIP = 11
        RIGHT_HIP = 12
        LEFT_KNEE = 13
        RIGHT_KNEE = 14
        LEFT_ANKLE = 15
        RIGHT_ANKLE = 16
        
        try:
            # Extract and validate keypoints
            kp = {
                'nose': keypoints[NOSE*3:(NOSE+1)*3],
                'left_shoulder': keypoints[LEFT_SHOULDER*3:(LEFT_SHOULDER+1)*3],
                'right_shoulder': keypoints[RIGHT_SHOULDER*3:(RIGHT_SHOULDER+1)*3],
                'left_hip': keypoints[LEFT_HIP*3:(LEFT_HIP+1)*3],
                'right_hip': keypoints[RIGHT_HIP*3:(RIGHT_HIP+1)*3],
                'left_knee': keypoints[LEFT_KNEE*3:(LEFT_KNEE+1)*3],
                'right_knee': keypoints[RIGHT_KNEE*3:(RIGHT_KNEE+1)*3],
                'left_ankle': keypoints[LEFT_ANKLE*3:(LEFT_ANKLE+1)*3],
                'right_ankle': keypoints[RIGHT_ANKLE*3:(RIGHT_ANKLE+1)*3]
            }
            
            # Check confidence of keypoints
            if any(point[2] < self.CONFIDENCE_THRESHOLD for point in kp.values()):
                return False, "low_confidence", ["Low confidence in keypoints"]

            # Get coordinates (x,y) for each keypoint
            nose = (kp['nose'][0], kp['nose'][1])
            ls = (kp['left_shoulder'][0], kp['left_shoulder'][1])
            rs = (kp['right_shoulder'][0], kp['right_shoulder'][1])
            lh = (kp['left_hip'][0], kp['left_hip'][1])
            rh = (kp['right_hip'][0], kp['right_hip'][1])
            lk = (kp['left_knee'][0], kp['left_knee'][1])
            rk = (kp['right_knee'][0], kp['right_knee'][1])
            la = (kp['left_ankle'][0], kp['left_ankle'][1])
            ra = (kp['right_ankle'][0], kp['right_ankle'][1])

            # 1. Height condition: shoulders near feet (as described in paper)
            torso_mid = ((lh[0] + rh[0])/2, (lh[1] + rh[1])/2)
            Lfactor = math.hypot(ls[0] - torso_mid[0], ls[1] - torso_mid[1])
            max_feet_y = max(la[1], ra[1])
            min_shoulder_y = min(ls[1], rs[1])
            height_cond = min_shoulder_y >= (max_feet_y - self.LENGTH_FACTOR_ALPHA * Lfactor)
            
            # 2. Speed calculation for rapid downward movement
            current_time = time.time()
            vertical_speed = 0
            current_min_y = min(ls[1], rs[1])
            
            if self.prev_shoulder_y is not None and self.prev_frame_time is not None:
                time_elapsed = current_time - self.prev_frame_time
                if time_elapsed > 0:
                    vertical_speed = (current_min_y - self.prev_shoulder_y) / time_elapsed
                    self.velocity_buffer.append(abs(vertical_speed))
            
            # Use smoothed velocity for more stable detection
            avg_speed = sum(self.velocity_buffer)/len(self.velocity_buffer) if self.velocity_buffer else 0
            speed_cond = avg_speed >= self.VELOCITY_THRESHOLD
            
            # 3. Angle conditions for leg collapse
            left_leg_angle = self.calculate_angle(lh, lk, la)
            right_leg_angle = self.calculate_angle(rh, rk, ra)
            leg_angle_cond = min(left_leg_angle, right_leg_angle) < self.ANGLE_THRESHOLD
            
            # 4. Torso orientation for horizontal posture
            torso_angle = self.calculate_torso_angle([ls, rs], [lh, rh])
            torso_cond = torso_angle > self.TORSO_ANGLE_THRESHOLD
            
            # 5. Body orientation ratio (width vs height)
            head_to_feet = abs(nose[1] - max_feet_y)
            body_width = abs(ls[0] - rs[0])
            orientation_ratio = body_width / (head_to_feet + 1e-6)
            aspect_cond = orientation_ratio > 0.8  # Similar to ORIENTATION_RATIO_THRESHOLD in paper
            
            # Count how many conditions are met
            conditions_met = sum([height_cond, speed_cond, leg_angle_cond, torso_cond, aspect_cond])
            
            # State machine logic as described in the paper
            if self.current_state == "normal":
                # Paper: "If 'shoulders_near_feet' in conditions and 'rapid_downward' movement"
                if height_cond and speed_cond:
                    self.current_state = "falling"
                    self.fall_start_time = current_time
            
            elif self.current_state == "falling":
                # Paper: "if 'horizontal_posture' in conditions and time < 1.0 seconds"
                if torso_cond and self.fall_start_time and (current_time - self.fall_start_time < 1.0):
                    self.current_state = "fallen"
                # Reset if not completed within timeframe
                elif current_time - self.fall_start_time > 1.0:
                    self.current_state = "normal"
            
            elif self.current_state == "fallen":
                # Stay in fallen state for some time before checking if person recovered
                if current_time - self.fall_start_time > 5.0:
                    if not height_cond and not torso_cond:
                        self.current_state = "normal"
            
            # Prepare conditions info for display
            conditions_info = []
            if height_cond:
                conditions_info.append("shoulders_near_feet")
            if speed_cond:
                conditions_info.append(f"rapid_downward_{avg_speed:.1f}px/s")
            if torso_cond:
                conditions_info.append("horizontal_posture")
            if leg_angle_cond:
                conditions_info.append(f"legs_collapsed_{min(left_leg_angle, right_leg_angle):.0f}°")
            if aspect_cond:
                conditions_info.append(f"orientation_ratio:{orientation_ratio:.2f}")
            
            # Fall detection logic - using both state machine and conditions count
            # Paper states "If 'shoulders_near_feet' in conditions and time < 1.0 seconds"
            is_fall = (self.current_state == "fallen") or (conditions_met >= 2 and height_cond)
            
            # Use buffer for temporal consistency (from shared code)
            self.fall_buffer.append(is_fall)
            final_detection = sum(self.fall_buffer) >= 1 if len(self.fall_buffer) >= 1 else is_fall
            
            if final_detection and self.current_state != "fallen":
                self.current_state = "fallen"
            
            # Update tracking variables
            self.prev_keypoints = {'left_shoulder': kp['left_shoulder'], 'right_shoulder': kp['right_shoulder']}
            self.prev_shoulder_y = current_min_y
            self.prev_frame_time = current_time
            
            return final_detection, self.current_state, conditions_info
            
        except Exception as e:
            print(f"Error in fall detection: {str(e)}")
            return False, "error", [f"Error: {str(e)}"]

    def reset(self):
        """Reset the detector state for a new video."""
        self.prev_keypoints = None
        self.velocity_buffer.clear()
        self.fall_buffer.clear()
        self.prev_frame_time = None
        self.fall_start_time = None
        self.prev_shoulder_y = None
        self.current_state = "normal"

print("FallDetector class defined")

FallDetector class defined


In [7]:
# Cell 4: Define utilities for annotation parsing
def parse_annotation(annotation_path):
    """Parse ground truth annotation file containing start and end frame numbers of falls."""
    try:
        with open(annotation_path, 'r') as f:
            lines = f.readlines()
            if len(lines) >= 2:
                return (int(lines[0].strip()), int(lines[1].strip()))
    except Exception as e:
        print(f"Error reading annotation file: {str(e)}")
        return None
    return None

def send_telegram_alert(bot_token, chat_id, message):
    """Send alert via Telegram when a fall is detected."""
    current_time = time.time()
    
    # Throttle alerts to avoid spam (max one alert per 2 minutes)
    if hasattr(send_telegram_alert, 'last_alert_time') and current_time - send_telegram_alert.last_alert_time < 120:
        return False
    
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "HTML"
    }
    
    try:
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            send_telegram_alert.last_alert_time = current_time
            print(f"Alert sent: {message}")
            return True
        else:
            print(f"Failed to send alert: {response.status_code} {response.text}")
    except Exception as e:
        print(f"Error sending Telegram alert: {str(e)}")
    
    return False

# Initialize the last alert time
send_telegram_alert.last_alert_time = 0

print("Utility functions defined")

Utility functions defined


In [8]:
# Cell 5: Define video processing function
def process_video(video_path, fall_detector, annotation_path=None, bot_token=None, chat_id=None, display=True):
    """
    Process video for fall detection using the FallDetector class.
    
    Args:
        video_path: Path to video file
        fall_detector: Instance of FallDetector class
        annotation_path: Path to ground truth annotation file (optional)
        bot_token: Telegram bot token (optional)
        chat_id: Telegram chat ID (optional)
        display: Whether to display frames (default: True)
    
    Returns:
        detected_frames: List of frames where falls were detected
    """
    # Reset the detector for new video
    fall_detector.reset()
    
    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return []

    # Get video properties
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Determine frame skip rate to achieve target FPS
    skip_frames = max(1, int(round(original_fps / fall_detector.TARGET_FPS)))
    
    # Initialize result variables
    detected_frames = []
    frame_counter = 0
    
    # Get ground truth annotation if available
    annotation_range = parse_annotation(annotation_path) if annotation_path else None

    print(f"Processing video: {video_path}")
    print(f"Original FPS: {original_fps}, Target FPS: {fall_detector.TARGET_FPS}, Skipping every {skip_frames} frames")
    
    # Process video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_counter += 1
        
        # Skip frames to achieve target FPS
        if frame_counter % skip_frames != 0:
            continue

        # Preprocess frame for YOLO model
        img = letterbox(frame, 640, stride=64, auto=True)[0]
        img_tensor = transforms.ToTensor()(img)
        img_tensor = torch.tensor(np.array([img_tensor.numpy()]))

        if torch.cuda.is_available():
            img_tensor = img_tensor.half().to(device)

        # Run inference
        with torch.no_grad():
            output, _ = model(img_tensor)
            output = non_max_suppression_kpt(output, 0.25, 0.65, 
                                          nc=model.yaml['nc'], 
                                          nkpt=model.yaml['nkpt'], 
                                          kpt_label=True)
            output = output_to_keypoint(output)

        # Visualization and fall detection
        display_frame = img.copy()
        display_frame = cv2.cvtColor(display_frame, cv2.COLOR_RGB2BGR)
        fall_detected = False
        current_state = "normal"
        conditions = []

        # Process detected persons
        if len(output) > 0:
            for idx in range(output.shape[0]):
                # Get keypoints for the person
                keypoints = output[idx, 7:].T
                
                # Draw skeleton on the frame
                plot_skeleton_kpts(display_frame, keypoints, 3)
                
                # Detect fall using the FallDetector
                is_fall, state, conds = fall_detector.detect_fall(keypoints)
                current_state = state
                conditions = conds
                
                if is_fall:
                    fall_detected = True
                    if frame_counter not in detected_frames:
                        detected_frames.append(frame_counter)
                        
                        # Send alert if Telegram credentials are provided
                        if bot_token and chat_id:
                            alert_msg = f"⚠️ <b>ALERT:</b> Fall detected!\nTime: {time.strftime('%H:%M:%S')}\nState: {current_state}\nConditions: {', '.join(conditions)}"
                            send_telegram_alert(bot_token, chat_id, alert_msg)

        # Draw detection status on frame
        color = (0, 0, 255) if fall_detected else (0, 255, 0)
        cv2.putText(display_frame, f"State: {current_state}", (20, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        # Draw conditions
        y_offset = 60
        for cond in conditions:
            cv2.putText(display_frame, cond, (20, y_offset), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            y_offset += 25

        # Draw ground truth if available
        if annotation_range:
            start, end = annotation_range
            gt_text = f"GT: {start}-{end} {'(FALL)' if start <= frame_counter <= end else ''}"
            cv2.putText(display_frame, gt_text, (20, y_offset+25), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        if display:
            # For Jupyter: display image directly
            rgb_frame = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(12, 8))
            plt.imshow(rgb_frame)
            plt.title(f"Frame {frame_counter} - State: {current_state}")
            plt.axis('off')
            plt.tight_layout()
            clear_output(wait=True)
            display(plt.gcf())
            plt.close()
            
            # Add small delay to simulate video
            time.sleep(0.05)

    # Release resources
    cap.release()
    print(f"Video processing complete. Detected falls in {len(detected_frames)} frames.")
    return detected_frames

In [9]:
# Cell 6: Define evaluation functions
def evaluate_videos(video_dir, label_dir, bot_token=None, chat_id=None):
    """
    Evaluate fall detection on a directory of videos with ground truth labels.
    
    Args:
        video_dir: Directory containing video files
        label_dir: Directory containing ground truth label files
        bot_token: Telegram bot token (optional)
        chat_id: Telegram chat ID (optional)
    
    Returns:
        metrics: Dictionary containing evaluation metrics
    """
    metrics = defaultdict(int)
    results = []
    
    # Create fall detector instance
    fall_detector = FallDetector()

    # Process each video file
    for video_file in os.listdir(video_dir):
        if not video_file.endswith(('.avi', '.mp4', '.mov')):
            continue

        video_path = os.path.join(video_dir, video_file)
        video_name = os.path.splitext(video_file)[0]
        label_path = os.path.join(label_dir, f"{video_name}.txt")
        
        print(f"\nProcessing video: {video_file}")
        
        # Check if annotation exists
        if not os.path.exists(label_path):
            print(f"Warning: No annotation found for {video_file}")
            continue

        # Process video with visualization
        detected_frames = process_video(video_path, fall_detector, label_path, bot_token, chat_id, display=False)
        
        # Update metrics based on detection results
        annotation_range = parse_annotation(label_path)
        gt_fall = annotation_range is not None
        true_detection = False
        
        if gt_fall and annotation_range and detected_frames:
            start, end = annotation_range
            
            # Check if any detected frame is within ground truth range
            true_detection = any(start <= frame <= end for frame in detected_frames)
        
        # Update confusion matrix counts
        if gt_fall:
            if true_detection:
                metrics['tp'] += 1  # True positive
            else:
                metrics['fn'] += 1  # False negative
        else:
            if detected_frames:
                metrics['fp'] += 1  # False positive
            else:
                metrics['tn'] += 1  # True negative
                
        # Record individual result
        results.append({
            'video': video_file,
            'gt_fall': gt_fall,
            'detected_fall': bool(detected_frames),
            'true_detection': true_detection if gt_fall else None,
            'frames_detected': detected_frames
        })

    return metrics, results

def calculate_metrics(metrics):
    """Calculate performance metrics from confusion matrix."""
    tp = metrics['tp']
    fp = metrics['fp']
    tn = metrics['tn']
    fn = metrics['fn']

    # Calculate derived metrics
    total = tp + tn + fp + fn
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'accuracy': round(accuracy * 100, 2),
        'precision': round(precision * 100, 2),
        'recall': round(recall * 100, 2),
        'specificity': round(specificity * 100, 2),
        'f1': round(f1 * 100, 2),
        'confusion_matrix': {
            'TP': tp,
            'FP': fp,
            'TN': tn,
            'FN': fn
        }
    }

print("Evaluation functions defined")

Evaluation functions defined


In [10]:
# Cell 7: Define webcam function for real-time demo
def demo_webcam(bot_token=None, chat_id=None):
    """Run fall detection on webcam feed in Jupyter notebook."""
    print("Starting fall detection on webcam feed...")
    fall_detector = FallDetector()
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Preprocess frame
            img = letterbox(frame, 640, stride=64, auto=True)[0]
            img_tensor = transforms.ToTensor()(img)
            img_tensor = torch.tensor(np.array([img_tensor.numpy()]))

            if torch.cuda.is_available():
                img_tensor = img_tensor.half().to(device)

            # Inference
            with torch.no_grad():
                output, _ = model(img_tensor)
                output = non_max_suppression_kpt(output, 0.25, 0.65, 
                                              nc=model.yaml['nc'], 
                                              nkpt=model.yaml['nkpt'], 
                                              kpt_label=True)
                output = output_to_keypoint(output)

            # Visualization
            display_frame = img.copy()
            display_frame = cv2.cvtColor(display_frame, cv2.COLOR_RGB2BGR)
            fall_detected = False
            current_state = "normal"
            conditions = []

            if len(output) > 0:
                for idx in range(output.shape[0]):
                    keypoints = output[idx, 7:].T
                    plot_skeleton_kpts(display_frame, keypoints, 3)
                    
                    is_fall, state, conds = fall_detector.detect_fall(keypoints)
                    current_state = state
                    conditions = conds
                    
                    if is_fall:
                        fall_detected = True
                        if bot_token and chat_id:
                            alert_msg = f"⚠️ <b>ALERT:</b> Fall detected in live feed!\nTime: {time.strftime('%H:%M:%S')}\nState: {current_state}\nConditions: {', '.join(conditions)}"
                            send_telegram_alert(bot_token, chat_id, alert_msg)

            # Draw detection status
            color = (0, 0, 255) if fall_detected else (0, 255, 0)
            cv2.putText(display_frame, f"State: {current_state}", (20, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Draw conditions
            y_offset = 60
            for cond in conditions:
                cv2.putText(display_frame, cond, (20, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                y_offset += 25

            # Privacy notice
            cv2.putText(display_frame, "Privacy-preserving: No data stored", 
                       (20, display_frame.shape[0]-20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            # For Jupyter: display image directly
            rgb_frame = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(12, 8))
            plt.imshow(rgb_frame)
            plt.title(f"Fall Detection - State: {current_state}")
            plt.axis('off')
            plt.tight_layout()
            clear_output(wait=True)
            display(plt.gcf())
            plt.close()
            
            # Add small delay
            time.sleep(0.05)
            
            # Check for keyboard interrupt
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    except KeyboardInterrupt:
        print("Stopping webcam capture...")
    finally:
        cap.release()
        print("Webcam released")

print("Webcam demo function defined")

Webcam demo function defined


In [11]:
# Cell 8: Examples using your Fall Dataset structure
# Set paths to your dataset
FALL_DATASET_ROOT = "../datasets/FallDataset"

# Example 1: Process a single test video file
def run_single_video_test():
    # Find the first video file in the test folder
    test_video_dir = os.path.join(FALL_DATASET_ROOT, "test/video")
    test_label_dir = os.path.join(FALL_DATASET_ROOT, "test/labels")
    
    # Create fall detector instance
    fall_detector = FallDetector()
    
    # List all video files
    video_files = [f for f in os.listdir(test_video_dir) 
                   if f.endswith(('.avi', '.mp4', '.mov'))]
    
    if not video_files:
        print(f"No video files found in {test_video_dir}")
        return
    
    # Use the first video file
    video_file = video_files[0]
    video_path = os.path.join(test_video_dir, video_file)
    
    # Check if label exists
    video_name = os.path.splitext(video_file)[0]
    label_path = os.path.join(test_label_dir, f"{video_name}.txt")
    if os.path.exists(label_path):
        print(f"Processing {video_file} with label")
        detected_frames = process_video(video_path, fall_detector, label_path)
    else:
        print(f"Processing {video_file} without label")
        detected_frames = process_video(video_path, fall_detector)
    
    print(f"Fall detected in frames: {detected_frames}")

# Example 2: Run the webcam demo
def run_webcam_demo():
    # Replace with your Telegram credentials if you want alerts
    telegram_bot_token = None  # "YOUR_BOT_TOKEN"
    telegram_chat_id = None    # "YOUR_CHAT_ID"
    
    demo_webcam(telegram_bot_token, telegram_chat_id)

# Example 3: Evaluate on your test dataset
def evaluate_fall_dataset():
    test_video_dir = os.path.join(FALL_DATASET_ROOT, "test/video")
    test_label_dir = os.path.join(FALL_DATASET_ROOT, "test/labels")
    
    print(f"Evaluating fall detection on test dataset:")
    print(f"Video directory: {test_video_dir}")
    print(f"Label directory: {test_label_dir}")
    
    metrics, results = evaluate_videos(test_video_dir, test_label_dir)
    evaluation = calculate_metrics(metrics)
    
    print("\nEvaluation Results:")
    print(f"Accuracy: {evaluation['accuracy']}%")
    print(f"Precision: {evaluation['precision']}%")
    print(f"Recall: {evaluation['recall']}%")
    print(f"Specificity: {evaluation['specificity']}%")
    print(f"F1-Score: {evaluation['f1']}%")
    print("\nConfusion Matrix:")
    print(f"True Positives (TP): {evaluation['confusion_matrix']['TP']}")
    print(f"False Positives (FP): {evaluation['confusion_matrix']['FP']}")
    print(f"True Negatives (TN): {evaluation['confusion_matrix']['TN']}")
    print(f"False Negatives (FN): {evaluation['confusion_matrix']['FN']}")
    
    # Print detailed results
    print("\nDetailed Results:")
    for result in results:
        status = "✓" if result['gt_fall'] == result['detected_fall'] else "✗"
        print(f"{status} {result['video']}: Ground Truth: {result['gt_fall']}, Detected: {result['detected_fall']}")

# Uncomment one of these to run the corresponding example
# run_single_video_test()
# run_webcam_demo()
evaluate_fall_dataset()

Evaluating fall detection on test dataset:
Video directory: ../datasets/FallDataset\test/video
Label directory: ../datasets/FallDataset\test/labels

Processing video: video (100).avi
Processing video: ../datasets/FallDataset\test/video\video (100).avi
Original FPS: 24.0003840061441, Target FPS: 25, Skipping every 1 frames


C:\Users\naufa\anaconda3\envs\cv_env\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Video processing complete. Detected falls in 0 frames.

Processing video: video (101).avi
Processing video: ../datasets/FallDataset\test/video\video (101).avi
Original FPS: 24.0003840061441, Target FPS: 25, Skipping every 1 frames
Video processing complete. Detected falls in 0 frames.

Processing video: video (102).avi
Processing video: ../datasets/FallDataset\test/video\video (102).avi
Original FPS: 24.0003840061441, Target FPS: 25, Skipping every 1 frames
Video processing complete. Detected falls in 7 frames.

Processing video: video (103).avi
Processing video: ../datasets/FallDataset\test/video\video (103).avi
Original FPS: 24.0003840061441, Target FPS: 25, Skipping every 1 frames
Video processing complete. Detected falls in 0 frames.

Processing video: video (104).avi
Processing video: ../datasets/FallDataset\test/video\video (104).avi
Original FPS: 24.0003840061441, Target FPS: 25, Skipping every 1 frames
Video processing complete. Detected falls in 0 frames.

Processing video: vi